In [1]:
# Loading libraries:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Ignoring future warnings printed for function sm.add_constant()
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Exercise 1: Perform the first pass regressions and tabulate the summary statistics

In [2]:
# Load Data
data = pd.read_excel('data.xlsx',header=1)
data

,Year,Market Index,A,B,C,D,E,F,G,H,I
0,1,29.65,33.88,-25.20,36.48,42.89,-39.89,39.67,74.57,40.22,90.19
1,2,-11.91,-49.87,24.70,-25.11,-54.39,44.92,-54.33,-79.76,-71.58,-26.64
2,3,14.73,65.14,-25.04,18.91,-39.86,-3.91,-5.69,26.73,14.49,18.14
3,4,27.68,14.46,-38.64,-23.31,-0.72,-3.21,92.39,-3.82,13.74,0.09
4,5,5.18,15.67,61.93,63.95,-32.82,44.26,-42.96,101.67,24.24,8.98
5,6,25.97,-32.17,44.94,-19.56,69.42,90.43,76.72,1.72,77.22,72.38
6,7,10.64,-31.55,-74.65,50.18,74.52,15.38,21.95,-43.95,-13.40,28.95
7,8,1.02,-23.79,47.02,-42.28,28.61,-17.64,28.83,98.01,28.12,39.41
8,9,18.82,-4.59,28.69,-0.54,2.32,42.36,18.93,-2.45,37.65,94.67
9,10,23.92,-8.03,48.61,23.65,26.26,-3.65,23.31,15.36,80.59,52.51


In [3]:
stock_excess_returns = data.iloc[:,2:]
stock_excess_returns


,A,B,C,D,E,F,G,H,I
0,33.88,-25.20,36.48,42.89,-39.89,39.67,74.57,40.22,90.19
1,-49.87,24.70,-25.11,-54.39,44.92,-54.33,-79.76,-71.58,-26.64
2,65.14,-25.04,18.91,-39.86,-3.91,-5.69,26.73,14.49,18.14
3,14.46,-38.64,-23.31,-0.72,-3.21,92.39,-3.82,13.74,0.09
4,15.67,61.93,63.95,-32.82,44.26,-42.96,101.67,24.24,8.98
5,-32.17,44.94,-19.56,69.42,90.43,76.72,1.72,77.22,72.38
6,-31.55,-74.65,50.18,74.52,15.38,21.95,-43.95,-13.40,28.95
7,-23.79,47.02,-42.28,28.61,-17.64,28.83,98.01,28.12,39.41
8,-4.59,28.69,-0.54,2.32,42.36,18.93,-2.45,37.65,94.67
9,-8.03,48.61,23.65,26.26,-3.65,23.31,15.36,80.59,52.51


In [4]:
averaged_stock_excess_returns = np.mean(stock_excess_returns)
averaged_stock_excess_returns

A     5.176667
B     4.190833
C     2.748333
D     6.150000
E     8.048333
F     9.903333
G    11.323333
H    13.110000
I    22.830000
dtype: float64

In [5]:
market_excess_returns = data['Market Index']
market_excess_returns

0     29.65
1    -11.91
2     14.73
3     27.68
4      5.18
5     25.97
6     10.64
7      1.02
8     18.82
9     23.92
10   -41.61
11    -6.64
Name: Market Index, dtype: float64

In [6]:
averaged_market_excess_returns = np.mean(market_excess_returns)
averaged_market_excess_returns

8.120833333333332

In [7]:
stocks = stock_excess_returns.columns
stocks

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'], dtype='object')

In [8]:
# Creating empty vectors for values
num_stocks = stock_excess_returns.shape[1]
a           = pd.Series([None]*num_stocks,index=stocks,dtype=float)
a_t_values  = pd.Series([None]*num_stocks,index=stocks,dtype=float)
b           = pd.Series([None]*num_stocks,index=stocks,dtype=float)
b_t_values  = pd.Series([None]*num_stocks,index=stocks,dtype=float)
r_square    = pd.Series([None]*num_stocks,index=stocks,dtype=float)

a

A   NaN
B   NaN
C   NaN
D   NaN
E   NaN
F   NaN
G   NaN
H   NaN
I   NaN
dtype: float64

In [9]:
x = market_excess_returns

# Adding constant
x = sm.add_constant(x,prepend=True)
    
x

,const,Market Index
0,1.0,29.65
1,1.0,-11.91
2,1.0,14.73
3,1.0,27.68
4,1.0,5.18
5,1.0,25.97
6,1.0,10.64
7,1.0,1.02
8,1.0,18.82
9,1.0,23.92


In [10]:
# First-pass regression
for stock in stocks:
    y = stock_excess_returns[stock]
    
    # Regression model
    # y = a + b*x + e
    model = sm.OLS(y,x)
    model = model.fit()

    # print(model.summary())

    a[stock]            = model.params['const']
    a_t_values[stock]   = model.tvalues['const']
    b[stock]            = model.params['Market Index']
    b_t_values[stock]   = model.tvalues['Market Index']
    r_square[stock]     = model.rsquared

In [11]:
# Inserting resutls into a data frame
first_pass_regression = pd.DataFrame()
first_pass_regression['a']          = a
first_pass_regression['a_t_values'] = a_t_values
first_pass_regression['b']          = b
first_pass_regression['b_t_values'] = b_t_values
first_pass_regression['r_square']   = r_square

first_pass_regression

,a,a_t_values,b,b_t_values,r_square
A,8.996946,0.726009,-0.470429,-0.814290,0.062184
B,-0.631102,-0.038668,0.593774,0.780379,0.057403
C,-0.636219,-0.055213,0.416774,0.775836,0.056775
D,-5.050588,-0.413886,1.379241,2.424471,0.370200
E,0.728982,0.053582,0.901305,1.421048,0.168010
F,-4.527576,-0.454789,1.777023,3.828906,0.594494
G,5.936470,0.325840,0.663339,0.780995,0.057489
H,-2.410250,-0.265253,1.911165,4.511642,0.670564
I,5.918236,0.636951,2.082516,4.807727,0.698015


## Exercise 2: Specify the hypotheses for a test of the second-pass regression of the SML
The hypotheses for the second-pass regression for the SML are: <br> 1) The intercept is zero. <br> 2) The slope is equal to the average return on the index portfolio. 

## Exercise 3: Perform the second-pass SML regression by regressing the average excess return for each portfolio on its betas

In [12]:
# averaged_stock_excess_returns = gamma_0 + gamma_1 * b + e
y = averaged_stock_excess_returns
x = first_pass_regression['b']
x = sm.add_constant(x,prepend=True)
x

,const,b
A,1.0,-0.470429
B,1.0,0.593774
C,1.0,0.416774
D,1.0,1.379241
E,1.0,0.901305
F,1.0,1.777023
G,1.0,0.663339
H,1.0,1.911165
I,1.0,2.082516


In [13]:
model = sm.OLS(y,x)
model = model.fit()

# print(model.summary())

print('gamma_0 = ' + str(model.params['const']))
print('gamma_1 = ' + str(model.params['b']))
print('sample-average risk premium: rM - rf = ' + str(averaged_market_excess_returns))

gamma_0 = 3.922998719398871
gamma_1 = 5.205334190864971
sample-average risk premium: rM - rf = 8.120833333333332


## Exercise 4: Summirize your test results and compare them to the results reported in the text
The intercept is too high (3.92% per year instead of 0)

The slope is too flat (5.21% instead of a predicted value equal to the sample-average risk premium: rM - rf = 8.12%).

## Exercise 5: Group the nine stocks into three portfolios, maximizing the dispersion of the betas of the three resultant portfolios. Repeat the test, and explain any changes to the results

In [14]:
first_pass_regression.sort_values('b', ascending=True)

,a,a_t_values,b,b_t_values,r_square
A,8.996946,0.726009,-0.470429,-0.814290,0.062184
C,-0.636219,-0.055213,0.416774,0.775836,0.056775
B,-0.631102,-0.038668,0.593774,0.780379,0.057403
G,5.936470,0.325840,0.663339,0.780995,0.057489
E,0.728982,0.053582,0.901305,1.421048,0.168010
D,-5.050588,-0.413886,1.379241,2.424471,0.370200
F,-4.527576,-0.454789,1.777023,3.828906,0.594494
H,-2.410250,-0.265253,1.911165,4.511642,0.670564
I,5.918236,0.636951,2.082516,4.807727,0.698015


In [15]:
portfolio_excess_returns = pd.DataFrame()
portfolio_excess_returns['ACB'] = np.mean(stock_excess_returns[['A','C','B']],1)
portfolio_excess_returns['GED'] = np.mean(stock_excess_returns[['G','E','D']],1)
portfolio_excess_returns['FHI'] = np.mean(stock_excess_returns[['F','H','I']],1)

portfolio_excess_returns

,ACB,GED,FHI
0,15.053333,25.856667,56.693333
1,-16.760000,-29.743333,-50.850000
2,19.670000,-5.680000,8.980000
3,-15.830000,-2.583333,35.406667
4,47.183333,37.703333,-3.246667
5,-2.263333,53.856667,75.440000
6,-18.673333,15.316667,12.500000
7,-6.350000,36.326667,32.120000
8,7.853333,14.076667,50.416667
9,21.410000,12.656667,52.136667


In [16]:
averaged_portfolio_excess_returns = np.mean(portfolio_excess_returns)
averaged_portfolio_excess_returns

ACB     4.038611
GED     8.507222
FHI    15.281111
dtype: float64

In [17]:
portfolios = portfolio_excess_returns.columns
portfolios

Index(['ACB', 'GED', 'FHI'], dtype='object')

In [18]:
# Creating empty vectors for values
num_portfolios = len(portfolios)
a           = pd.Series([None]*num_portfolios,index=portfolios,dtype=float)
a_t_values  = pd.Series([None]*num_portfolios,index=portfolios,dtype=float)
b           = pd.Series([None]*num_portfolios,index=portfolios,dtype=float)
b_t_values  = pd.Series([None]*num_portfolios,index=portfolios,dtype=float)
r_square    = pd.Series([None]*num_portfolios,index=portfolios,dtype=float)
a

ACB   NaN
GED   NaN
FHI   NaN
dtype: float64

In [19]:
x = market_excess_returns

# Adding constant
x = sm.add_constant(x,prepend=True)

x

,const,Market Index
0,1.0,29.65
1,1.0,-11.91
2,1.0,14.73
3,1.0,27.68
4,1.0,5.18
5,1.0,25.97
6,1.0,10.64
7,1.0,1.02
8,1.0,18.82
9,1.0,23.92


In [20]:
# First-pass regression
for portfolio in portfolios:
    y = portfolio_excess_returns[portfolio]

    # Regression
    # y = a + b*x + e
    model = sm.OLS(y,x)
    model = model.fit()

    # print(model.summary())

    a[portfolio]            = model.params['const']
    a_t_values[portfolio]   = model.tvalues['const']
    b[portfolio]            = model.params['Market Index']
    b_t_values[portfolio]   = model.tvalues['Market Index']
    r_square[portfolio]     = model.rsquared

first_pass_regression = pd.DataFrame()
first_pass_regression['a']          = a
first_pass_regression['a_t_values'] = a_t_values
first_pass_regression['b']          = b
first_pass_regression['b_t_values'] = b_t_values
first_pass_regression['r_square']   = r_square

first_pass_regression

,a,a_t_values,b,b_t_values,r_square
ACB,2.576541,0.416032,0.180039,0.623584,0.037430
GED,0.538288,0.077182,0.981295,3.018145,0.476692
FHI,-0.339864,-0.056226,1.923568,6.826155,0.823310


**Second-pass SML regression** <br> Perform the second-pass SML regression by regressing the average excess return for each portfolio on its betas

In [21]:
# averaged_portfolio_excess_returns = gamma_0 + gamma_1 * b + e
y = averaged_portfolio_excess_returns
x = first_pass_regression['b']
x = sm.add_constant(x,prepend=True)
model = sm.OLS(y,x)
model = model.fit()

# print(model.summary())

print('gamma_0 = ' + str(model.params['const']))
print('gamma_1 = ' + str(model.params['b']))
print('sample-average risk premium: rM - rf = ' + str(averaged_market_excess_returns))

gamma_0 = 2.622881506147568
gamma_1 = 6.469669670397844
sample-average risk premium: rM - rf = 8.120833333333332


The intercept is too high (2.62% per year instead of 0)

The slope is too flat (6.47% instead of a predicted value equal to the sample-average risk premium: rM - rf = 8.12%). 

## Exercise 8: Perform the first-pass regressions as did Chen, Roll and Ross and tabulate the relevant summary statistics.

In [22]:
change_in_factor_value =  pd.Series([
    -9.84,
    6.46,
    16.12,
    -16.51,
    17.82,
    -13.31,
    -3.52,
    8.43,
    8.23,
    7.06,
    -15.74,
    2.03],name='Change in factor value')

change_in_factor_value

0     -9.84
1      6.46
2     16.12
3    -16.51
4     17.82
5    -13.31
6     -3.52
7      8.43
8      8.23
9      7.06
10   -15.74
11     2.03
Name: Change in factor value, dtype: float64

In [23]:
stocks = stock_excess_returns.columns
stocks

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'], dtype='object')

In [24]:
# Creating empty vectors for values
num_stocks = stock_excess_returns.shape[1]
a               = pd.Series([None]*num_stocks,index=stocks,dtype=float)
a_t_values      = pd.Series([None]*num_stocks,index=stocks,dtype=float)
b_M             = pd.Series([None]*num_stocks,index=stocks,dtype=float)
b_M_t_values    = pd.Series([None]*num_stocks,index=stocks,dtype=float)
b_F             = pd.Series([None]*num_stocks,index=stocks,dtype=float)
b_F_t_values    = pd.Series([None]*num_stocks,index=stocks,dtype=float)
r_square        = pd.Series([None]*num_stocks,index=stocks,dtype=float)
a

A   NaN
B   NaN
C   NaN
D   NaN
E   NaN
F   NaN
G   NaN
H   NaN
I   NaN
dtype: float64

In [25]:
x = pd.concat([market_excess_returns, change_in_factor_value],axis=1)

# Adding constant
x = sm.add_constant(x,prepend=True)

x

,const,Market Index,Change in factor value
0,1.0,29.65,-9.84
1,1.0,-11.91,6.46
2,1.0,14.73,16.12
3,1.0,27.68,-16.51
4,1.0,5.18,17.82
5,1.0,25.97,-13.31
6,1.0,10.64,-3.52
7,1.0,1.02,8.43
8,1.0,18.82,8.23
9,1.0,23.92,7.06


In [26]:
# First-pass regression
for stock in stocks:
    y = stock_excess_returns[stock]
    
    # Regression model
    # y = a + b_M * x_M + b_F * x_F + e
    model = sm.OLS(y,x)
    model = model.fit()

    # print(model.summary())

    a[stock]                = model.params['const']
    a_t_values[stock]       = model.tvalues['const']
    b_M[stock]              = model.params['Market Index']
    b_M_t_values[stock]     = model.tvalues['Market Index']
    b_F[stock]              = model.params['Change in factor value']
    b_F_t_values[stock]     = model.tvalues['Change in factor value']
    r_square[stock]         = model.rsquared

# Inserting resutls into a data frame
first_pass_regression = pd.DataFrame()
first_pass_regression['a']              = a
first_pass_regression['a_t_values']     = a_t_values
first_pass_regression['b_M']            = b_M
first_pass_regression['b_M_t_values']   = b_M_t_values
first_pass_regression['b_F']            = b_F
first_pass_regression['b_F_t_values']   = b_F_t_values
first_pass_regression['r_square']       = r_square

first_pass_regression

,a,a_t_values,b_M,b_M_t_values,b_F,b_F_t_values,r_square
A,9.185439,0.706936,-0.467771,-0.772891,-0.348680,-0.336559,0.073840
B,-1.890295,-0.133097,0.576015,0.870716,2.329298,2.056915,0.358821
C,-0.996881,-0.084243,0.411688,0.746899,0.667165,0.707092,0.106416
D,-4.484849,-0.370381,1.387220,2.459521,-1.046525,-1.083936,0.442924
E,0.171960,0.012558,0.893450,1.400824,1.030398,0.943772,0.242935
F,-3.473850,-0.524161,1.791884,5.804534,-1.949219,-3.688647,0.838559
G,5.316517,0.285606,0.654596,0.754948,1.146811,0.772653,0.116119
H,-2.641257,-0.280415,1.907907,4.348625,0.427326,0.568988,0.682003
I,5.656420,0.589711,2.078824,4.652854,0.484315,0.633254,0.710897


## Exercise 9: Specify the hypothesis for a test of a second-pass regression for the two-factor SML

The hypotheses for the second-pass regression for the two-factor SML are:

1) The intercept is zero.

2) The market-index slope coefficient equals the market-index average return.

3) The factor slope coefficient equals the average return on the factor.

(Note that the first two hypotheses are the same as those for the single factor model.)

## Exercise 10: Do the data suggest a two-factor economy?

In [27]:
averaged_stock_excess_returns = np.mean(stock_excess_returns)
averaged_stock_excess_returns

A     5.176667
B     4.190833
C     2.748333
D     6.150000
E     8.048333
F     9.903333
G    11.323333
H    13.110000
I    22.830000
dtype: float64

In [28]:
x = pd.concat([first_pass_regression['b_M'], first_pass_regression['b_F']],axis=1)
x = sm.add_constant(x,prepend=True)
x

,const,b_M,b_F
A,1.0,-0.467771,-0.348680
B,1.0,0.576015,2.329298
C,1.0,0.411688,0.667165
D,1.0,1.387220,-1.046525
E,1.0,0.893450,1.030398
F,1.0,1.791884,-1.949219
G,1.0,0.654596,1.146811
H,1.0,1.907907,0.427326
I,1.0,2.078824,0.484315


In [29]:
y = averaged_stock_excess_returns

model = sm.OLS(y,x)
model = model.fit()

# Model coefficients
model.params

const    3.354409
b_M      5.533135
b_F      0.802378
dtype: float64

In [30]:
print('Average return on the factor: {}'.format(np.mean(change_in_factor_value)))

Average return on the factor: 0.6024999999999997


In [31]:
# t values
model.tvalues

const    1.162945
b_M      2.558741
b_F      0.563503
dtype: float64

The intercept is too high (3.35% per year instead of 0) <br> However, not significantly greater than zero (t-stat less than 2)

The slope to the market portfolio is too flat (5.53% instead of a predicted value equal to the sample-average risk premium: rM - rf = 8.12%).

The slope to the factor is too steep (0.8% instead of a predicted value equal to the sample-average: 0.6%).
